In [347]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

In [348]:
# Constants

gammac = 1.4
gammah = 1.33

cpa = 1.005
cp = 1.148
R = 287

In [349]:
# Compressor Values

Ca4 = 231.16 # m/s
Ca5 = Ca4
N = 208.05 # Rev/s
deltaTocomp = 439.58 # K

In [350]:
To4 = 2275 # K
polyt = 0.92
#flowcoeff Greater than 0.78
#loadcoeff Cannot Exceed 3.3
mfahot = 40 # kg/s
mfthot = mfahot + (mfahot*.02)

In [351]:
# Turbine PARAMETERS

htratioturbine = 0.74

In [352]:
deltaToturb = (deltaTocomp*cpa*0.99)/cp

T4 = To4-((Ca4**2)/(2*cp*1000))

po4 = 15 # bar
p4 = po4*(T4/To4)**(gammah/(gammah-1))

density4 = (p4*100000)/(R*T4)

print(density4)

2.2268738031901054


In [353]:
A4 = mfthot/(density4*Ca4)

M4 = Ca4/(np.sqrt(1.333*R*To4))

print("Turbine Inlet Area = %3.4f m^2" % (A4))

print("Turbine Inlet Mach Number = %3.2f" % (M4))

Turbine Inlet Area = 0.0793 m^2
Turbine Inlet Mach Number = 0.25


In [354]:
r4t = np.sqrt(mfthot/(np.pi*density4*Ca4*(1-(htratioturbine**2))))

r4h = htratioturbine*r4t

r4m = (r4t+r4h)/2

print("r4t = %3.3f m" % (r4t))
print("rm = %3.3f m" % (r4m))
print("r4h = %3.3f m" % (r4h))

r4t = 0.236 m
rm = 0.205 m
r4h = 0.175 m


In [355]:
To5 = To4 - deltaToturb

T5 = To5 - (Ca5**2)/(2*cp*1000)

po5 = po4*((To5/To4)**((gammah*polyt)/(gammah-1)))

p5 = po5*(T5/To5)**(gammah/(gammah-1))

density5 = (p5*100000)/(R*T5)

A5 = mfthot/(density5*Ca5)

print("Turbine Exhaust Area = %3.4f m^2" % (A5))

Turbine Exhaust Area = 0.1310 m^2


In [356]:
# constant mean radius
#bladeheight5 = A5/(2*np.pi*rm)
#
#r5t = rm+(bladeheight5/2)
#
#r5h = rm-(bladeheight5/2)


# constant hub radius
r5h = r4h
r5t = np.sqrt(A5/np.pi + (r5h)**2)
r5m = (r5t+r5h)/2

print("r5t = %3.3f m" % (r5t))
print("r5m = %3.3f m" % (r5m))
print("r5h = %3.3f m" % (r5h))

r5t = 0.269 m
r5m = 0.222 m
r5h = 0.175 m


In [357]:
U5h = r5h*(2*np.pi*N)

print(U5h)

flow5 = Ca5/U5h

print(flow5)

bladeloading = 3.3

deltaTos = (bladeloading*(U5h**2))/(2*cp*1000)

print(deltaTos)

stageest = deltaToturb/deltaTos

print("Estimation of %3.1f Stages" % (stageest))

228.4383471772574
1.011914167898574
75.00324865921372
Estimation of 5.1 Stages


In [358]:
U4t = 2*np.pi*r4t*N
U4m = 2*np.pi*r4m*N
U4h = 2*np.pi*r4h*N

In [359]:
stageactual = 6
# Values per stage: alpha2, alpha3, beta2, beta3, r, deltaT, T3, U, Cw2, Cw3, V2, V3, pr, po3, bladeloading, flowcoeff
stagevals = np.zeros((stageactual+1,3,17))
stagevals[0,:,:] = np.array([[0,0,0,0,0,0,To4,U4h,Ca4,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4m,Ca4,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4t,Ca4,0,0,0,0,0,po4,0,0]])

In [360]:
def initialvals4(deltaTocomp,Ca4,To4,po4,htratioturbine):
    deltaToturb = (deltaTocomp*cpa*0.99)/cp

    T4 = To4-((Ca4**2)/(2*cp*1000))

    po4 = 15 # bar
    p4 = po4*(T4/To4)**(gammah/(gammah-1))

    density4 = (p4*100000)/(R*T4)

    A4 = mfthot/(density4*Ca4)

    M4 = Ca4/(np.sqrt(1.333*R*To4))

    r4t = np.sqrt(mfthot/(np.pi*density4*Ca4*(1-(htratioturbine**2))))

    r4h = htratioturbine*r4t

    r4m = (r4t+r4h)/2

    return deltaToturb,r4t,r4h,r4m

In [361]:
def initialvals5(deltaToturb,Ca5,To4,po4,r4h):
    To5 = To4 - deltaToturb

    T5 = To5 - (Ca5**2)/(2*cp*1000)

    po5 = po4*((To5/To4)**((gammah*polyt)/(gammah-1)))

    p5 = po5*(T5/To5)**(gammah/(gammah-1))

    density5 = (p5*100000)/(R*T5)

    A5 = mfthot/(density5*Ca5)

    r5h = r4h

    r5t = np.sqrt(A5/np.pi+(r5h)**2)

    r5m = (r5t+r5h)/2

    return r5t,r5h,r5m

In [362]:
def estimatevals(r5h,N,Ca5,bladeloading,deltaToturb):
    U5h = r5h*(2*np.pi*N)

    flow5 = Ca5/U5h

    deltaTos = (bladeloading*(U5h**2))/(2*cp*1000)

    stageest = deltaToturb/deltaTos

    return stageest

$$
\psi = \frac{2 c_p \Delta T_{0s}}{U^2} \\
\phi = \frac{C_a}{U} \\
tan \beta_2 = \frac{1}{2 \phi} (\frac{\psi}{2} - 2\Lambda) \\
tan \beta_3 = \frac{1}{2 \phi} (\frac{\psi}{2} + 2\Lambda) \\
$$

In [363]:
def returnstage(deltaTo,U,Ca,DOR):
    bladeloading = (2*cp*1000*deltaTo)/(U**2)
    flow = Ca/U
    beta2 = np.degrees(np.arctan((1/(2*flow))*((bladeloading/2)-(2*DOR))))
    beta3 = np.degrees(np.arctan((1/(2*flow))*((bladeloading/2)+(2*DOR))))
    alpha2 = np.degrees(np.arctan(np.tan(np.radians(beta2))+(1/flow)))
    alpha3 = np.degrees(np.arctan((np.tan(np.radians(beta3))-(1/flow))))

    Cw2 = Ca*np.tan(np.radians(alpha2))
    Cw3 = Ca*np.tan(np.radians(alpha3))

    V2 = Ca/(np.cos(np.radians(beta2)))
    V3 = Ca/(np.cos(np.radians(beta3)))

    return beta2,beta3,alpha2,alpha3,Cw2,Cw3,V2,V3,bladeloading,flow

In [364]:
def iterate(stagevals,stageactual,deltaTosact,r4t,r4m,r4h,N,Ca):
    i = 1

    # This runs through the params for each stage and saves them to an array
    while i < stageactual+1:
        if i == 0:
            DOR = 0.5
        if i == 1:
            DOR = 0.55
        if i == 2 or i == 3:
            DOR = 0.6
        if i == 4 or i == 5:
            DOR = 0.7
        else:
            DOR = 0.8

        rtstage = ((r5t-r4t)/(stageactual-1))*i+r4t 
        rmstage = ((r5m-r4m)/(stageactual-1))*i+r4m
        rhstage = ((r5h-r4h)/(stageactual-1))*i+r4h

        Utstage = 2*np.pi*rtstage*N
        Umstage = 2*np.pi*rmstage*N
        Uhstage = 2*np.pi*rhstage*N

        stageTo3 = stagevals[i-1,0,6] - deltaTosact

        pr = ((stageTo3/stagevals[i-1,0,6])**((gammah*polyt)/(gammah-1)))

        # DO NOT TOUCH THE COMMENTED FUNCTION UNLESS THE FIRST STAGE VALUES ARE FUCKED UP

        """if i == 1:
            deltawhirlt = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Utstage))
            Cw2t = deltawhirlt - Cw1t
            stagebeta1tip = np.degrees(np.arctan(Utstage/Castage))
            stagebeta2tip = np.degrees(np.arctan((Utstage-Cw2t)/Castage))
            stagealpha1tip = 0
            stagealpha2tip = np.degrees(np.arctan((Cw2t)/Castage))

            V1t = Castage/(np.cos(np.radians(stagebeta1tip)))
            dehallert = np.cos(np.radians(stagebeta1tip))/np.cos(np.radians(stagebeta2tip))
            V2t = V1t*dehallert

            deltawhirlm = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Umcomp))
            Cw2m = deltawhirlm - Cw1m
            stagebeta1mean = np.degrees(np.arctan(Umcomp/Castage))
            stagebeta2mean = np.degrees(np.arctan((Umcomp-Cw2m)/Castage))
            stagealpha1mean = 0
            stagealpha2mean = np.degrees(np.arctan((Cw2m)/Castage))

            V1m = Castage/(np.cos(np.radians(stagebeta1mean)))
            dehallerm = np.cos(np.radians(stagebeta1mean))/np.cos(np.radians(stagebeta2mean))
            V2m = V1m*dehallerm

            deltawhirlh = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Uhcomp))
            Cw2h = deltawhirlh - Cw1h
            stagebeta1hub = np.degrees(np.arctan(Uhcomp/Castage))
            stagebeta2hub = np.degrees(np.arctan((Uhcomp-Cw2h)/Castage))
            stagealpha1hub = 0
            stagealpha2hub = np.degrees(np.arctan((Cw2h)/Castage))

            V1h = Castage/(np.cos(np.radians(stagebeta1hub)))
            dehallerh = np.cos(np.radians(stagebeta1hub))/np.cos(np.radians(stagebeta2hub))
            V2h = V1h*dehallerh
        else: """
        stagebeta2tip,stagebeta3tip,stagealpha2tip,stagealpha3tip,Cw2t,Cw3t,V2t,V3t,bladelt,flowt = returnstage(deltaTosact,Utstage,Ca,DOR)

        stagebeta2mean,stagebeta3mean,stagealpha2mean,stagealpha3mean,Cw2m,Cw3m,V2m,V3m,bladelm,flowm = returnstage(deltaTosact,Umstage,Ca,DOR)

        stagebeta2hub,stagebeta3hub,stagealpha2hub,stagealpha3hub,Cw2h,Cw3h,V2h,V3h,bladelh,flowh = returnstage(deltaTosact,Uhstage,Ca,DOR)

        #pr = (1+((polyc*(((Umstage*Castage)/(cpa*1000))*(np.tan(np.radians(stagebeta1mean))-np.tan(np.radians(stagebeta2mean)))))/(stagevals[i-1,0,6])))**(gammac/(gammac-1))
        stagepo3 = stagevals[i-1,0,14]*pr

        stagevals[i,:,:] = np.array([[stagealpha2hub,stagealpha3hub,stagebeta2hub,stagebeta3hub,rhstage,deltaTosact,stageTo3,Uhstage,Ca,Cw2h,Cw3h,V2h,V3h,pr,stagepo3,bladelh,flowh],[stagealpha2mean,stagealpha3mean,stagebeta2mean,stagebeta3mean,rmstage,deltaTosact,stageTo3,Umstage,Ca,Cw2m,Cw3m,V2m,V3m,pr,stagepo3,bladelm,flowm],[stagealpha2tip,stagealpha3tip,stagebeta2tip,stagebeta3tip,rtstage,deltaTosact,stageTo3,Utstage,Ca,Cw2t,Cw3t,V2t,V3t,pr,stagepo3,bladelt,flowt]])

        i += 1

    return stagevals

In [365]:
deltaTosact = deltaToturb/stageactual
print(deltaTosact)
stagevals = iterate(stagevals,stageactual,deltaTosact,r4t,r4m,r4h,N,Ca4)

63.495952526132406


In [366]:
def valtable(location,vals):
    if location == 0:
        print("Hub Values:")
    elif location == 1:
        print("Mean Values:")
    elif location == 2:
        print("Tip Values:")
    
    num = 0
    stages = ["Inlet"]

    while num < len(vals[:,location,4])-1:
        stages.append("Stage "+str(num+1))
        num += 1

    pd.set_option("display.precision", 4)
    tabletry = pd.DataFrame({"Alpha 2": vals[:,location,0],"Alpha 3": vals[:,location,1],"Beta 2": vals[:,location,2],"Beta 3": vals[:,location,3],"Radius": vals[:,location,4],"Delta To": vals[:,location,5],"To2": vals[:,location,6],"U": vals[:,location,7],"Ca": vals[:,location,8],"Cw1": vals[:,location,9],"Cw2": vals[:,location,10],"V1": vals[:,location,11],"V2": vals[:,location,12],"PR": vals[:,location,13],"po2": vals[:,location,14],"Blade Loading": vals[:,location,15],"Flow": vals[:,location,16]},index=stages)

    return tabletry

In [367]:
valtable(0,stagevals)

Hub Values:


,Alpha 2,Alpha 3,Beta 2,Beta 3,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,2275.0000,228.4383,231.16,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,41.6002,26.2229,-5.7321,55.9681,0.1748,63.496,2211.5040,228.4383,231.16,205.2348,113.8594,232.3217,413.0408,0.9004,13.5054,2.7937,1.0119
Stage 2,41.6002,26.2229,-5.7321,55.9681,0.1748,63.496,2148.0081,228.4383,231.16,205.2348,113.8594,232.3217,413.0408,0.8976,12.1226,2.7937,1.0119
Stage 3,41.6002,26.2229,-5.7321,55.9681,0.1748,63.496,2084.5121,228.4383,231.16,205.2348,113.8594,232.3217,413.0408,0.8947,10.8462,2.7937,1.0119
Stage 4,44.6156,21.4913,-0.0892,54.1101,0.1748,63.496,2021.0162,228.4383,231.16,228.0786,91.0156,231.1603,394.3168,0.8916,9.6708,2.7937,1.0119
Stage 5,44.6156,21.4913,-0.0892,54.1101,0.1748,63.496,1957.5202,228.4383,231.16,228.0786,91.0156,231.1603,394.3168,0.8884,8.5913,2.7937,1.0119
Stage 6,41.6002,26.2229,-5.7321,55.9681,0.1748,63.496,1894.0243,228.4383,231.16,205.2348,113.8594,232.3217,413.0408,0.8849,7.6026,2.7937,1.0119


In [368]:
valtable(1,stagevals)

Mean Values:


,Alpha 2,Alpha 3,Beta 2,Beta 3,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,2275.0000,268.5694,231.16,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,39.1438,18.8722,-20.1196,56.6959,0.2087,63.496,2211.5040,272.8346,231.16,188.1522,79.0183,246.1830,420.9934,0.9004,13.5054,1.9585,0.8473
Stage 2,38.9640,18.2241,-21.3054,56.7970,0.2120,63.496,2148.0081,277.0998,231.16,186.9490,76.1091,248.1171,422.1273,0.8976,12.1226,1.8987,0.8342
Stage 3,38.7926,17.5851,-22.4592,56.9021,0.2152,63.496,2084.5121,281.3649,231.16,185.8082,73.2623,250.1320,423.3148,0.8947,10.8462,1.8415,0.8216
Stage 4,42.6975,10.2767,-17.3772,54.7878,0.2185,63.496,2021.0162,285.6301,231.16,213.2900,41.9119,242.2149,400.8974,0.8916,9.6708,1.7869,0.8093
Stage 5,42.6174,9.5174,-18.4684,54.8789,0.2218,63.496,1957.5202,289.8953,231.16,212.6922,38.7550,243.7114,401.8034,0.8884,8.5913,1.7347,0.7974
Stage 6,38.3265,15.7212,-25.7358,57.2392,0.2250,63.496,1894.0243,294.1604,231.16,182.7328,65.0686,256.6146,427.1773,0.8849,7.6026,1.6848,0.7858


In [369]:
valtable(2,stagevals)

Tip Values:


,Alpha 2,Alpha 3,Beta 2,Beta 3,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,2275.0000,308.7005,231.16,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,37.6496,12.5465,-30.9999,57.9122,0.2427,63.496,2211.5040,317.2308,231.16,178.3363,51.4439,269.6788,435.1506,0.9004,13.5054,1.4487,0.7287
Stage 2,37.4467,11.4284,-32.7571,58.1771,0.2492,63.496,2148.0081,325.7612,231.16,177.0338,46.7294,274.8723,438.3881,0.8976,12.1226,1.3738,0.7096
Stage 3,37.2668,10.3383,-34.4216,58.4486,0.2557,63.496,2084.5121,334.2915,231.16,175.8849,42.1683,280.2277,441.7657,0.8947,10.8462,1.3046,0.6915
Stage 4,42.1397,0.8593,-30.0374,56.2755,0.2623,63.496,2021.0162,342.8219,231.16,209.1603,3.4672,267.0213,416.3545,0.8916,9.6708,1.2405,0.6743
Stage 5,42.1367,-0.4147,-31.6006,56.5328,0.2688,63.496,1957.5202,351.3522,231.16,209.1383,-1.6731,271.4033,419.1783,0.8884,8.5913,1.1810,0.6579
Stage 6,36.8509,7.2232,-38.9165,59.2911,0.2753,63.496,1894.0243,359.8826,231.16,173.2503,29.2973,297.0968,452.6543,0.8849,7.6026,1.1256,0.6423


In [370]:
#check = False

In [371]:
"""end = 0
Ca = 150
while not check:
    HTratio = 0.3
    while HTratio < .8:
        N = 220
        while N < 250:
            deltaToturb,r4t,r4h,r4m = initialvals4(deltaTocomp,Ca,To4,po4,HTratio)
            r5t,r5h,r5m = initialvals5(deltaToturb,Ca,To4,po4,r4h)
            stageest = estimatevals(r5h,N,Ca,3.3,deltaToturb)
            stageactuala = np.ceil(stageest).astype('int32')
            # Values per stage: alpha2, alpha3, beta2, beta3, r, deltaT, T3, U, Cw2, Cw3, V2, V3, pr, po3, bladeloading, flowcoeff
            stagevalsa = np.zeros((stageactuala+1,3,17))
            stagevalsa[0,:,:] = np.array([[0,0,0,0,0,0,To4,U4h,Ca,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4m,Ca,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4t,Ca,0,0,0,0,0,po4,0,0]])
            deltaTosact = deltaToturb/stageactuala
            stagevalsa = iterate(stagevalsa,stageactuala,deltaTosact,r4t,r4m,r4h,N,Ca)
            
            if stagevalsa[1,0,15] < 3.3 and stagevalsa[stageactuala,2,16] > 0.78 and stagevalsa[1,0,16] > 0.78 and stagevalsa[stageactuala,2,7] < 290:
                print("Stagecount:"+str(stageactuala))
                print(len(stagevalsa[:,0,4]))
                print(stagevalsa)
                check = True
            N += 1
        HTratio += .1

    Ca += 0.1
    if end > 10000:
        check = True
        print("BAH")
    end += 1

print(HTratio)
print(N)
print(Ca)
print(end)"""

'end = 0\nCa = 150\nwhile not check:\n    HTratio = 0.3\n    while HTratio < .8:\n        N = 220\n        while N < 250:\n            deltaToturb,r4t,r4h,r4m = initialvals4(deltaTocomp,Ca,To4,po4,HTratio)\n            r5t,r5h,r5m = initialvals5(deltaToturb,Ca,To4,po4,r4h)\n            stageest = estimatevals(r5h,N,Ca,3.3,deltaToturb)\n            stageactuala = np.ceil(stageest).astype(\'int32\')\n            # Values per stage: alpha2, alpha3, beta2, beta3, r, deltaT, T3, U, Cw2, Cw3, V2, V3, pr, po3, bladeloading, flowcoeff\n            stagevalsa = np.zeros((stageactuala+1,3,17))\n            stagevalsa[0,:,:] = np.array([[0,0,0,0,0,0,To4,U4h,Ca,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4m,Ca,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4t,Ca,0,0,0,0,0,po4,0,0]])\n            deltaTosact = deltaToturb/stageactuala\n            stagevalsa = iterate(stagevalsa,stageactuala,deltaTosact,r4t,r4m,r4h,N,Ca)\n            \n            if stagevalsa[1,0,15] < 3.3 and stagevalsa[stageactuala,2,16] > 0.78